In [1]:
pip install pyomo pandas openpyxl


  Obtaining dependency information for pyomo from https://files.pythonhosted.org/packages/d1/dc/139e49cee5b003e32bd38d222dfd8ced549078c98809f88f736abd3d7650/Pyomo-6.8.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/a8/44/d9502bf0ed197ba9bf1103c9867d5904ddcaf869e52329787fc54ed70cc8/pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Using cached pandas-2.2.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata (89 kB)
  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for ply from https://files.pythonhosted.org/packages/a3/58/35da89ee790598a0700ea49b2a66594140f44dec458c07e8e3d4979137fc/ply-3.11-py2.py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/pack

In [2]:
import pandas as pd
from pyomo.environ import *
from datetime import datetime, timedelta

In [488]:
# Load data from Excel
teams_df = pd.read_excel('schedule_data.xlsx', sheet_name='Teams')

# Extract divisions from columns
divisions = teams_df.columns.tolist()

# Extract teams per division
division_teams = {}
for div in divisions:
    division_teams[div] = teams_df[div].dropna().tolist()

# Combine all teams
all_teams = []
for teams in division_teams.values():
    all_teams.extend(teams)

In [489]:
division_teams

{'FPL': ['Empire Lions',
  'Greater Helsinki Markhors',
  'The J Team Jaguars',
  'Helsinki Titans',
  'TreCC Amperes',
  'Vantaa Legends',
  'SKK Rapids',
  'BTCC Thundercats'],
 'NLD1': ['NWSC Warriors',
  'AWCC Storm Riders',
  'FinCricket Club',
  'SKK Smashers',
  'Turku Hawks',
  'Shaurya T20A',
  'Gymkhana Gladiators',
  'BTCC Avengers']}

In [490]:
# Load ground availability data
grounds_df = pd.read_excel('schedule_data.xlsx', sheet_name='Grounds')
grounds_df['Date'] = pd.to_datetime(grounds_df['Date'], format='%d-%m-%Y')  # Adjust date format as needed
dates_sorted = sorted(grounds_df['Date'].unique().tolist())

# Extract ground names (excluding 'Date' column)
grounds = list(grounds_df.columns)
grounds.remove('Date')

In [491]:
grounds_df

,Date,Kerava,Tikkurila,Rajakylä,Käpylä
0,2024-08-22,1,0,1,1
1,2024-08-21,1,0,1,1
2,2024-08-20,1,0,1,1
3,2024-08-19,1,0,1,1
4,2024-08-18,1,0,1,1
...,...,...,...,...,...
65,2024-05-31,0,0,1,1
66,2024-05-29,0,0,1,1
67,2024-05-28,0,0,1,1
68,2024-05-27,0,0,1,1


In [492]:
# Define ground preferences for NLD 1
ground_preferences = {
    'Kerava': 1,
    'Tikkurila': 2,
    'Rajakylä': 3,
    'Käpylä': 4
}

# For FPL, only Kerava is allowed
fpl_allowed_grounds = ['Kerava']

# For NLD 1, all grounds are allowed, with preferences
nld1_allowed_grounds = ['Kerava', 'Tikkurila', 'Rajakylä', 'Käpylä']

In [493]:
# Function to determine if a date is a weekend
def is_weekend(date):
    return 1 if date.weekday() >= 5 else 0  # 1 for Saturday/Sunday, 0 otherwise

# Add 'IsWeekend' column
grounds_df['IsWeekend'] = grounds_df['Date'].apply(is_weekend)

# Initialize ground capacity
ground_capacity = {}
for idx, row in grounds_df.iterrows():
    date = row['Date']
    is_weekend_flag = row['IsWeekend']
    for ground in grounds:
        if row[ground] == 1:
            capacity = 3 if is_weekend_flag else 1
            ground_capacity[(ground, date)] = capacity
        else:
            ground_capacity[(ground, date)] = 0


In [494]:
# Generate matches within each division
matches = {}
for div in divisions:
    teams = division_teams[div]
    matches[div] = [(h, a) for h in teams for a in teams if h != a]


In [495]:
matches


{'FPL': [('Empire Lions', 'Greater Helsinki Markhors'),
  ('Empire Lions', 'The J Team Jaguars'),
  ('Empire Lions', 'Helsinki Titans'),
  ('Empire Lions', 'TreCC Amperes'),
  ('Empire Lions', 'Vantaa Legends'),
  ('Empire Lions', 'SKK Rapids'),
  ('Empire Lions', 'BTCC Thundercats'),
  ('Greater Helsinki Markhors', 'Empire Lions'),
  ('Greater Helsinki Markhors', 'The J Team Jaguars'),
  ('Greater Helsinki Markhors', 'Helsinki Titans'),
  ('Greater Helsinki Markhors', 'TreCC Amperes'),
  ('Greater Helsinki Markhors', 'Vantaa Legends'),
  ('Greater Helsinki Markhors', 'SKK Rapids'),
  ('Greater Helsinki Markhors', 'BTCC Thundercats'),
  ('The J Team Jaguars', 'Empire Lions'),
  ('The J Team Jaguars', 'Greater Helsinki Markhors'),
  ('The J Team Jaguars', 'Helsinki Titans'),
  ('The J Team Jaguars', 'TreCC Amperes'),
  ('The J Team Jaguars', 'Vantaa Legends'),
  ('The J Team Jaguars', 'SKK Rapids'),
  ('The J Team Jaguars', 'BTCC Thundercats'),
  ('Helsinki Titans', 'Empire Lions'),
  (

In [541]:
model = ConcreteModel()

# Sets
model.Divisions = Set(initialize=divisions)
model.Teams = Set(initialize=all_teams)
model.DivisionTeams = Set(model.Divisions, initialize=division_teams)
model.Dates = Set(initialize=dates_sorted)
model.Grounds = Set(initialize=grounds)
model.Matches = Set(dimen=3)  # (Division, HomeTeam, AwayTeam)

# Flatten matches and associate with divisions
all_matches = []
for div in divisions:
    for match in matches[div]:
        all_matches.append((div, match[0], match[1]))
model.AllMatches = Set(initialize=all_matches, dimen=3)

# Ground Availability Parameter
model.GroundAvailability = Param(model.Grounds, model.Dates, initialize=ground_capacity, default=0)

# Determine if a date is weekend
date_weekend = {date: is_weekend(date) for date in dates_sorted}
model.IsWeekend = Param(model.Dates, initialize=date_weekend, within=Binary)

# Ground preferences for NLD 1 (lower number means higher preference)
model.GroundPreference = Param(model.Grounds, initialize=ground_preferences, default=100)


# Map each date to its ISO week number
date_to_week = {date: date.isocalendar()[1] for date in dates_sorted}

# Create a set of unique weeks
weeks = sorted(set(date_to_week.values()))
model.Weeks = Set(initialize=weeks)

# Parameter to map dates to weeks
model.DateToWeek = Param(model.Dates, initialize=date_to_week, within=NonNegativeIntegers)


# Decision variable: y[div, h, a, d, g] = 1 if match is scheduled on date d at ground g
model.y = Var(model.AllMatches, model.Dates, model.Grounds, domain=Binary)

# Auxiliary variable: u[t, d, g] = 1 if team t uses ground g on day d
model.u = Var(model.Teams, model.Dates, model.Grounds, domain=Binary)

# Binary variable: z_day[d] = 1 if any match is scheduled on date d
model.z_day = Var(model.Dates, domain=Binary)



In [ ]:
def match_scheduled_once_rule(model, div, h, a):
    return sum(model.y[div, h, a, d, g] for d in model.Dates for g in model.Grounds) == 1
model.MatchScheduledOnce = Constraint(model.AllMatches, rule=match_scheduled_once_rule)


def ground_capacity_rule(model, g, d):
    return sum(model.y[div, h, a, d, g] for div, h, a in model.AllMatches) <= model.GroundAvailability[g, d]
model.GroundCapacityConstraint = Constraint(model.Grounds, model.Dates, rule=ground_capacity_rule)

def team_two_match_per_day_rule(model, t, d):
    return sum(model.y[div, h, a, d, g] 
               for div, h, a in model.AllMatches 
               if h == t or a == t 
               for g in model.Grounds) <= 2
model.TeamTwoMatchPerDay = Constraint(model.Teams, model.Dates, rule=team_two_match_per_day_rule)



In [543]:
# Constraint: If team t has a match on day d at ground g, then u[t, d, g] must be 1
def link_u_to_y_rule(model, t, d, g):
    return sum(model.y[div, h, a, d, g] 
               for div, h, a in model.AllMatches 
               if (h == t or a == t)) <= model.u[t, d, g] * 2
model.LinkUtoY = Constraint(model.Teams, model.Dates, model.Grounds, rule=link_u_to_y_rule)

# Constraint: A team can use at most one ground per day
def one_ground_per_day_rule(model, t, d):
    return sum(model.u[t, d, g] for g in model.Grounds) <= 1
model.OneGroundPerDay = Constraint(model.Teams, model.Dates, rule=one_ground_per_day_rule)


In [544]:
# 4. No Team Plays More Than Two Matches Per Week
def team_max_two_matches_per_week_rule(model, t, w):
    return sum(model.y[div, h, a, d, g] 
               for div, h, a in model.AllMatches 
               if h == t or a == t 
               for d in model.Dates if model.DateToWeek[d] == w 
               for g in model.Grounds) <= 2
model.TeamMaxTwoMatchesPerWeek = Constraint(model.Teams, model.Weeks, rule=team_max_two_matches_per_week_rule)


In [545]:
# FPL teams play only at Kerava
def fpl_ground_rule(model, div, h, a, d, g):
    if div == 'FPL':
        if g == 'Kerava':
            return model.y[div, h, a, d, g] <= 1 
        else:
            return model.y[div, h, a, d, g] == 0
    else:
        return Constraint.Skip
model.FPLGroundConstraint = Constraint(model.AllMatches, model.Dates, model.Grounds, rule=fpl_ground_rule)


In [546]:
# Define special teams per division
special_teams = {
    'FPL': 'TreCC Amperes',    # Replace with actual team name
    'NLD1': 'Turku Hawks'  # Replace with actual team name
}

# Constraint: Special teams only play on weekends
def special_team_weekend_rule(model, div, h, a, d, g):
    if h == special_teams[div] or a == special_teams[div]:
        return model.y[div, h, a, d, g] <= model.IsWeekend[d]
    else:
        return Constraint.Skip
model.SpecialTeamWeekendConstraint = Constraint(model.AllMatches, model.Dates, model.Grounds, rule=special_team_weekend_rule)

# Constraint: Special teams can have up to 2 matches per weekend day
def special_team_max_two_matches_per_day_rule(model, div, d):
    if model.IsWeekend[d] == 1:
        return sum(
            model.y[div, h, a, d, g]
            for _div, h, a in model.AllMatches
            if _div == div and (h == special_teams[div] or a == special_teams[div])
            for g in model.Grounds
        ) <= 2
    else:
        return Constraint.Skip



In [547]:
model.Objective = Objective(expr=0, sense=minimize)


In [548]:
# Choose the solver
solver = SolverFactory('glpk',executable='/usr/local/bin/glpsol')  

# Solve the model
result = solver.solve(model, tee=True)


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmpeu19l_w8.glpk.raw
 --wglp /var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp0fe0ynra.glpk.glp
 --cpxlp /var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmph6zcepm8.pyomo.lp
Reading problem data from '/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmph6zcepm8.pyomo.lp'...
/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmph6zcepm8.pyomo.lp:411780: warning: lower bound of variable 'x4' redefined
/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmph6zcepm8.pyomo.lp:411780: warning: upper bound of variable 'x4' redefined
30856 rows, 35841 columns, 283360 non-zeros
35840 integer variables, all of which are binary
447620 lines were read
Writing problem data to '/var/folders/23/m666xnxn3yb59qvbhl4ddhk80000gn/T/tmp0fe0ynra.glpk.glp'...
369157 lines were written
GLPK Integer Optimizer 5.0
30856 rows, 35841 columns, 283360 non-zeros
35840 integer variables,

In [550]:
# Check if the solution is feasible
if (result.solver.status == SolverStatus.ok) and (result.solver.termination_condition == TerminationCondition.optimal):
    # Retrieve the schedule
    schedule = []
    for div, h, a in model.AllMatches:
        for d in model.Dates:
            for g in model.Grounds:
                if value(model.y[div, h, a, d, g]) == 1:
                    schedule.append({
                        'Division': div,
                        'HomeTeam': h,
                        'AwayTeam': a,
                        'Date': d.strftime('%d-%m-%Y'),
                        'Ground': g
                    })
    schedule_df = pd.DataFrame(schedule)
    
    # Sort the schedule by date and division
    schedule_df.sort_values(by=['Date', 'Division'], inplace=True)
    
    # Reset index
    schedule_df.reset_index(drop=True, inplace=True)
    
    # Display the schedule
    print(schedule_df)
    
    # Optionally, export to Excel
    schedule_df.to_excel('schedule_output.xlsx', index=False)
else:
    print('No feasible solution found.')
    print('Solver Status:', result.solver.status)
    print('Termination Condition:', result.solver.termination_condition)


    Division                   HomeTeam                   AwayTeam  \
0        FPL            Helsinki Titans              TreCC Amperes   
1        FPL             Vantaa Legends  Greater Helsinki Markhors   
2        FPL                 SKK Rapids              TreCC Amperes   
3       NLD1               Shaurya T20A                Turku Hawks   
4        FPL                 SKK Rapids         The J Team Jaguars   
..       ...                        ...                        ...   
107     NLD1              BTCC Avengers                Turku Hawks   
108      FPL  Greater Helsinki Markhors            Helsinki Titans   
109      FPL              TreCC Amperes  Greater Helsinki Markhors   
110      FPL           BTCC Thundercats             Vantaa Legends   
111     NLD1               SKK Smashers              NWSC Warriors   

           Date    Ground  
0    01-06-2024    Kerava  
1    01-06-2024    Kerava  
2    01-06-2024    Kerava  
3    01-06-2024  Rajakylä  
4    01-07-2024    